In [1]:
# Dependencies
import requests
from dotenv import load_dotenv
import os
import pandas as pd

https://api.stlouisfed.org/fred/category/series?category_id=125&api_key=abcdefghijklmnopqrstuvwxyz123456
https://fred.stlouisfed.org/docs/api/fred/category_series.html#Description 

In [2]:
# Set environment variables from the .env in the local environment
def key_check(key_path=None):
    try:
        reply=load_dotenv(key_path,verbose=True,override=True)   
        assert reply , 'Dotenv is not found'
        fred_api_key = os.getenv("FRED_API_KEY")
        assert fred_api_key is not None, 'FRED_API_KEY not found in .env file'
        responce=requests.get(f'https://api.stlouisfed.org/fred/category/series?category_id=125&api_key={fred_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate tmdb_api_key {fred_api_key} code {responce.status_code}'
    except Exception as e:
        # Handle potential errors in loading .env or missing API keys
        print(f'An error occurred: {e}')
        return(False)
    else:
        print('All keys loaded correctly')
        return (True)
my_env_path='C:\SRC\AI\ProjectOne_Team2\Frank_playground\keys.env'
if key_check(my_env_path):
    fred_api_key = os.getenv("FRED_API_KEY")
else:
    print ('fix Keys and rerun')

All keys loaded correctly


In [3]:
# read the file into a list
list_of_series_df = pd.read_csv('C:\SRC\AI\ProjectOne_Team2\Frank_playground\list_of_series.csv')

In [4]:
#  Set the base URL
series_id=''
url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_api_key}&file_type=json")

In [5]:
def create_filename(row):
    # Join the desired fields with underscores
    filename = '_'.join(row[['country', 'state', 'county', 'series_Desc']])
    # Replace spaces with underscores and remove any problematic characters
    filename = filename.replace(' ', '_').replace('/', '_')
    return filename

In [6]:

for index, row in list_of_series_df.iterrows():
    series_id=list_of_series_df.loc[index,'series_id']
    url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_api_key}&file_type=json")
    series = requests.get(url).json()
    # Create an empty list to store the reviews
    series_list =[]
    for item in series["observations"]:
        if item['value'] != '.':
            series_list.append(item)      
    # Clean Data
    series_df = pd.DataFrame(series_list)
    series_df = (
        series_df
        .drop(series_df.columns[[0, 1]], axis=1)
        .assign(date=lambda df: pd.to_datetime(df['date']))
        .set_index('date')
        .dropna()
    )
    # CREATE A FILENAME AND WRITE THE DATA TO THE FILE.
    filename = create_filename(row)
    list_of_series_df['filename']=list_of_series_df.apply(create_filename, axis=1)
    file_path = f"../resources/{filename}.csv"  # Construct file path with .csv extension
    series_df.to_csv(file_path, index=True)

In [7]:
list_of_series_df.to_csv('C:/SRC/AI/ProjectOne_Team2/Frank_playground/output_filename.csv', index=False)
display (list_of_series_df)

,country,state,county,series_Desc,series_id,filename
0,United States,TX,Collin,House Price Index,ATNHPIUS48085A,United_States_TX_Collin_House_Price_Index
1,United States,TX,Dallas,House Price Index,ATNHPIUS48113A,United_States_TX_Dallas_House_Price_Index
2,United States,TX,Denton,House Price Index,ATNHPIUS48121A,United_States_TX_Denton_House_Price_Index
3,United States,TX,Collin,Household Income,MHITX48085A052NCEN,United_States_TX_Collin_Household_Income
4,United States,TX,Dallas,Household Income,MHITX48113A052NCEN,United_States_TX_Dallas_Household_Income
5,United States,TX,Denton,Household Income,MHITX48121A052NCEN,United_States_TX_Denton_Household_Income
6,United States,TX,Collin,Resident Population,TXCOLL5POP,United_States_TX_Collin_Resident_Population
7,United States,TX,Dallas,Resident Population,TXDALL3POP,United_States_TX_Dallas_Resident_Population
8,United States,TX,Denton,Resident Population,TXDENT2POP,United_States_TX_Denton_Resident_Population
